# Libraries

In [ ]:
# Essential Libraries
import os  # File management
import shutil  # Copying/moving files
import zipfile  # Handling compressed files

# Data Processing
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import matplotlib.image as mpimg  # displaying images
import seaborn as sns

# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import Huber

# Serialization & Model Saving
import joblib  # Loading scalers
import pickle  # Loading Python objects

# Import Test Data

In [ ]:
#Load the dataset
df = pd.read_csv("../../csv/20250301_df_19_clean_filtered.csv")

#  Generate 'file_path' based on 'subset' column
df["file_path"] = df.apply(
    lambda row: f"/content/augmented_images_test/{row['variety']}/{row['file_name']}"
    if row["subset"] == "test"
    else f"/content/augmented_images_train/{row['variety']}/{row['file_name']}",
    axis=1
)

# Drop unnecessary columns
df.drop(columns=["simp_amount", "rank"], inplace=True)

#  Rename & Reorder columns
df.rename(columns={"adjusted_weight": "weight"}, inplace=True)
df = df[["file_path", "label", "packed", "crowd", "amount", "weight", "subset"]]

#  Create Test Set
df_test = df[df["subset"] == "test"].drop(columns=["subset"])

#  Modify file paths for test images
df_test["file_path"] = df_test["file_path"].str.replace("/content", "../test", regex=False)

#  Display first 2 rows
df_test.head(2)

## Create dictionary to identify fruits by their label

In [ ]:
# List of fruit/vegetable varieties (same order as for the segmentation model (YOLO))
fruits_only = [
    "carrot_carrot", "apple_red_delicious", "tomato_pink", "cucumber_long", "banana_yellow",
    "apple_granny", "apple_fuji", "pepper_sweet_red", "orange_orange", "onion_white",
    "apple_ligol", "lime_lime", "avocado_hass", "apple_golden", "kiwi_kiwi",
    "tomato_cherry_red", "pepper_sweet_green", "pepper_sweet_yellow", "lemon_yellow"
]

# Create dictionary with matching indices
varieties_dict = {idx: variety for idx, variety in enumerate(fruits_only)}

print(varieties_dict)

# Import Mean Weight and Scaler

In [ ]:
# Define paths
mean_weight_path = "../model/weight_scaler/mean_weight.pkl"
scaler_path = "../model/weight_scaler/weight_scaler_sample1.pkl"
amount_scaler_path = '../model/weight_scaler/amount_scaler_sample1.pkl'


# Load the mean weight
with open(mean_weight_path, "rb") as f:
    mean_weight = pickle.load(f)

# Load the weight scaler (input) and the count scaler (output)
weight_scaler = joblib.load(scaler_path)
count_scaler = joblib.load(amount_scaler_path)

# Convert to NumPy array & reshape, convert to DataFrame for consistency before scaling
test_weights = df_test["weight"].values.reshape(-1, 1)  
test_weights_df = pd.DataFrame(test_weights, columns=["weight"])

# Scale the weights using the loaded scaler
test_weights_scaled = weight_scaler.transform(test_weights_df)

print(" Test weights successfully loaded and scaled!")

# Load Test Data

In [ ]:
#  Load test dataset features
test_image_paths = df_test["file_path"].values
test_weights = df_test["weight"].values
test_labels = df_test["label"].values
test_amounts = df_test["amount"].values
test_packed = df_test["packed"].values
test_crowd = df_test["crowd"].values

#  Function to load and preprocess a single image
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # Decode JPEG
    image = tf.image.resize(image, (224, 224))  # Resize for model input
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]
    return image

#  Load images using `map()`
test_images = np.array([load_image(img).numpy() for img in test_image_paths])

print(f"Loaded {len(test_images)} test images successfully!")

# Load Trained Model

In [ ]:
# Load the trained model with custom Huber loss
model_path = "../model/models/best_model_EfficientNet.keras"
model = load_model(model_path, custom_objects={"Huber": Huber})

print("Model loaded successfully!")

# I. Prediction (including the weight feature)

## Get Model Prediction

In [ ]:
# Get model predictions

predictions = model.predict([test_images, test_weights_scaled])
batch_size = 32  # Adjust based on available RAM
num_batches = len(test_images) // batch_size + 1  # Total batches

predictions = []

for i in range(num_batches):
    batch_images = test_images[i * batch_size: (i + 1) * batch_size]
    batch_weights = test_weights_scaled[i * batch_size: (i + 1) * batch_size]
    
    batch_preds = model.predict([batch_images, batch_weights], verbose=0)
    predictions.append(batch_preds)

# Combine batch predictions
predictions = [np.concatenate([batch[i] for batch in predictions], axis=0) for i in range(len(predictions[0]))]

print("Testing completed successfully!")

## Extract predictions for each output

In [ ]:
# Predictions

fruit_class_preds = np.argmax(predictions[0], axis=1)  # Convert softmax to class index
fruit_count_preds = predictions[1].flatten()  # Regression output
bagged_preds = (predictions[2].flatten() > 0.5).astype(int)  # Convert sigmoid output to binary
crowded_preds = (predictions[3].flatten() > 0.5).astype(int)  # Convert sigmoid output to binary

# Evaluation metrics

fruit_class_accuracy = accuracy_score(test_labels, fruit_class_preds)
bagged_accuracy = accuracy_score(test_packed, bagged_preds)
crowded_accuracy = accuracy_score(test_crowd, crowded_preds)
huber_loss_mean = tf.losses.huber(test_amounts, fruit_count_preds, delta=1.0).numpy()

print("Fruit Classification Accuracy:", round(fruit_class_accuracy, 2))
print("Huber Loss:", round(huber_loss_mean,2))
print("Bagged Accuracy:", round(bagged_accuracy,2))
print("Crowded Accuracy:", round(crowded_accuracy,2))

## Create metrics dataframe

In [ ]:
# Scale the count results back to their original form. 

fruit_count_preds = np.array(fruit_count_preds).reshape(-1, 1)
fruit_count_preds_original = count_scaler.inverse_transform(fruit_count_preds).flatten()

# Create a dataframe for comparison
results_df = pd.DataFrame({
    "actual_fruit_class": test_labels,
    "predicted_fruit_class": fruit_class_preds,
    "actual_fruit_count": test_amounts,
    "predicted_fruit_count": fruit_count_preds_original,
    "actual_bagged": test_packed,
    "predicted_bagged": bagged_preds,
    "actual_crowded": test_crowd,
    "predicted_crowded": crowded_preds
})

# Add the error counts and fruit names

results_df['count_error'] = results_df['actual_fruit_count'] - results_df['predicted_fruit_count']
results_df['predicted_fruit_name'] = results_df['predicted_fruit_class'].map(varieties_dict)
results_df['actual_fruit_name'] = results_df['actual_fruit_class'].map(varieties_dict)

# Save results to a CSV for further analysis
results_df.to_csv("test_results_withWeight.csv", index=False)

print("Results saved successfully!")

## Load metrics dataframe

In [ ]:
# Load the uploaded test results CSV file
file_path = "test_results_withWeight.csv"
results_df = pd.read_csv(file_path)

# Display the first few rows of the dataset
results_df.head()

## Visualisations

### Confusion Matrix for Fruit Classification

In [ ]:
# Extract unique fruit names sorted for consistent label mapping
fruit_names = sorted(results_df["actual_fruit_name"].unique())

# Compute confusion matrix
cm = confusion_matrix(results_df["actual_fruit_name"], results_df["predicted_fruit_name"], labels=fruit_names)

# Plot heatmap with fruit name labels
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="BuGn", xticklabels=fruit_names, yticklabels=fruit_names)

plt.xlabel("Predicted Fruit")
plt.ylabel("Actual Fruit")
plt.title("Confusion Matrix for Fruit Classification (with weight of fruits)")
plt.xticks(rotation=90)  # Rotate labels for better readability
plt.yticks(rotation=0)
plt.show()

### Distribution of Errors in Fruit Count Predictions

In [ ]:
# Distribution of Errors in Fruit Count Predictions

plt.figure(figsize=(8, 5))
sns.histplot(results_df["count_error"], bins=30, kde=True, color="green")
plt.axvline(0, color="red", linestyle="dashed", label="Perfect Prediction (Error = 0)")
plt.xlabel("Prediction Error (Actual - Predicted)")
plt.ylabel("Frequency")
plt.title("Distribution of Errors in Fruit Count Predictions (with weight of fruits)")
plt.legend()
plt.show()

### Regression Performance for Fruit Count (Scatter Plot)

In [ ]:
# Regression Performance for Fruit Count (Scatter Plot)

plt.figure(figsize=(8, 5))
plt.scatter(results_df["actual_fruit_count"], results_df["predicted_fruit_count"], alpha=0.5, color='green')
plt.xlabel("Actual Fruit Count")
plt.ylabel("Predicted Fruit Count")
plt.title("Regression Performance for Fruit Count (with weight of fruits)")
plt.axline((0, 0), slope=1, color="red", linestyle="dashed", label="Perfect Prediction Line")
plt.legend()
plt.show()

### Binary Classification Performance: ROC Curve for Bagged

In [ ]:
# Binary Classification Performance (ROC Curve for Bagged & Crowded)
plt.figure(figsize=(12, 5))

# Bagged ROC Curve
plt.subplot(1, 2, 1)
fpr_bagged, tpr_bagged, _ = roc_curve(results_df["actual_bagged"], results_df["predicted_bagged"])
roc_auc_bagged = auc(fpr_bagged, tpr_bagged)
plt.plot(fpr_bagged, tpr_bagged, label=f"AUC = {roc_auc_bagged:.2f}")
plt.plot([0, 1], [0, 1], linestyle="dashed", color="red")  # Random chance line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for Bagged Prediction")
plt.legend()

plt.tight_layout()
plt.show()

### Binary Classification Performance: ROC Curve for Crowded

In [ ]:
# Crowded ROC Curve
plt.subplot(1, 2, 2)
fpr_crowded, tpr_crowded, _ = roc_curve(results_df["actual_crowded"], results_df["predicted_crowded"])
roc_auc_crowded = auc(fpr_crowded, tpr_crowded)
plt.plot(fpr_crowded, tpr_crowded, label=f"AUC = {roc_auc_crowded:.2f}")
plt.plot([0, 1], [0, 1], linestyle="dashed", color="red")  # Random chance line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for Crowded Prediction")
plt.legend()

plt.tight_layout()
plt.show()

# II. Predictions (image only)

## Set weight to 0

In [ ]:
# Create a placeholder weight array filled with the mean weight
num_samples = len(test_images)  # Match the number of test samples
placeholder_weights = np.full((num_samples, 1), mean_weight)  # Shape (num_samples, 1)

# Scale the placeholder weights
placeholder_weights_scaled = weight_scaler.transform(placeholder_weights)

batch_size = 32  
predictions_weight0 = []  # Store batch predictions

# Process test images in batches
for i in range(0, len(test_images), batch_size):
    batch_images = test_images[i: i + batch_size]
    batch_weights = placeholder_weights_scaled[i: i + batch_size]  # Using placeholder weights

    batch_preds = model.predict([batch_images, batch_weights], verbose=0)
    predictions_weight0.append(batch_preds)

# Combine batch predictions
predictions_weight0 = [np.concatenate([batch[i] for batch in predictions_weight0], axis=0) for i in range(len(predictions_weight0[0]))]

print("Testing completed successfully without requiring weight input!")

## Extract predictions for each output

In [ ]:
# Extract predictions for each output
fruit_class_preds = np.argmax(predictions_weight0[0], axis=1)  # Convert softmax to class index
fruit_count_preds = predictions_weight0[1].flatten()  # Regression output
bagged_preds = (predictions_weight0[2].flatten() > 0.5).astype(int)  # Convert sigmoid output to binary
crowded_preds = (predictions_weight0[3].flatten() > 0.5).astype(int)  # Convert sigmoid output to binary

# Evaluation metrics

fruit_class_accuracy = accuracy_score(test_labels, fruit_class_preds)
bagged_accuracy = accuracy_score(test_packed, bagged_preds)
crowded_accuracy = accuracy_score(test_crowd, crowded_preds)
huber_loss_mean = tf.losses.huber(test_amounts, fruit_count_preds, delta=1.0).numpy()

print("Fruit Classification Accuracy:", round(fruit_class_accuracy, 2))
print("Huber Loss:", round(huber_loss_mean,2))
print("Bagged Accuracy:", round(bagged_accuracy,2))
print("Crowded Accuracy:", round(crowded_accuracy,2))



## Create metrics dataframe

In [ ]:
# Scale the count results back to their original form. 

fruit_count_preds = np.array(fruit_count_preds).reshape(-1, 1)
fruit_count_preds_original = count_scaler.inverse_transform(fruit_count_preds).flatten()

# Create a dataframe for comparison
results_df_weight0 = pd.DataFrame({
    "actual_fruit_class": test_labels,  # True labels from test set
    "predicted_fruit_class": fruit_class_preds,
    "actual_fruit_count": test_amounts,
    "predicted_fruit_count": fruit_count_preds,
    "actual_bagged": test_packed,
    "predicted_bagged": bagged_preds,
    "actual_crowded": test_crowd,
    "predicted_crowded": crowded_preds
})

# Add the error counts and fruit names

results_df_weight0['count_error'] = results_df_weight0['actual_fruit_count'] - results_df_weight0['predicted_fruit_count']
results_df_weight0['predicted_fruit_name'] = results_df_weight0['predicted_fruit_class'].map(varieties_dict)
results_df_weight0['actual_fruit_name'] = results_df_weight0['actual_fruit_class'].map(varieties_dict)

# Save results to a CSV for further analysis
results_df_weight0.to_csv("test_results_weight0.csv", index=False)

print("Results saved successfully!")

## Load metric dataframe

In [ ]:
# Load the uploaded test results CSV file
file_path = "test_results_weight0.csv"
results_df_0 = pd.read_csv(file_path)

# Display the first few rows of the dataset
results_df_0.head()

## Visualisations

### Confusion Matrix for Fruit Classification

In [ ]:
# Extract unique fruit names sorted for consistent label mapping
fruit_names = sorted(results_df_0["actual_fruit_name"].unique())

# Compute confusion matrix
cm = confusion_matrix(results_df_0["actual_fruit_name"], results_df_0["predicted_fruit_name"], labels=fruit_names)

# Plot heatmap with fruit name labels
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="BuGn", xticklabels=fruit_names, yticklabels=fruit_names)

plt.xlabel("Predicted Fruit")
plt.ylabel("Actual Fruit")
plt.title("Confusion Matrix for Fruit Classification (only image as input)")
plt.xticks(rotation=90)  # Rotate labels for better readability
plt.yticks(rotation=0)
plt.show()

### Distribution of Errors in Fruit Count Predictions

In [ ]:
# Distribution of Errors in Fruit Count Predictions

plt.figure(figsize=(8, 5))
sns.histplot(results_df_0["count_error"], bins=30, kde=True, color="green")
plt.axvline(0, color="red", linestyle="dashed", label="Perfect Prediction (Error = 0)")
plt.xlabel("Prediction Error (Actual - Predicted)")
plt.ylabel("Frequency")
plt.title("Distribution of Errors in Fruit Count Predictions (only image as input)")
plt.legend()
plt.show()

### Regression Performance for Fruit Count (Scatter Plot)

In [ ]:
# Regression Performance for Fruit Count (Scatter Plot)

plt.figure(figsize=(8, 5))
plt.scatter(results_df_0["actual_fruit_count"], results_df_0["predicted_fruit_count"], alpha=0.5, color='green')
plt.xlabel("Actual Fruit Count")
plt.ylabel("Predicted Fruit Count")
plt.title("Regression Performance for Fruit Count (only image as input)")
plt.axline((0, 0), slope=1, color="red", linestyle="dashed", label="Perfect Prediction Line")
plt.legend()
plt.show()

### Binary Classification Performance: ROC Curve for Bagged

In [ ]:
# Binary Classification Performance (ROC Curve for Bagged & Crowded)
plt.figure(figsize=(12, 5))

# Bagged ROC Curve
plt.subplot(1, 2, 1)
fpr_bagged, tpr_bagged, _ = roc_curve(results_df_0["actual_bagged"], results_df_0["predicted_bagged"])
roc_auc_bagged = auc(fpr_bagged, tpr_bagged)
plt.plot(fpr_bagged, tpr_bagged, label=f"AUC = {roc_auc_bagged:.2f}")
plt.plot([0, 1], [0, 1], linestyle="dashed", color="red")  # Random chance line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for Bagged Prediction (only image as input)")
plt.legend()

plt.tight_layout()
plt.show()

### Binary Classification Performance: ROC Curve for Crowded

In [ ]:
# Crowded ROC Curve
plt.subplot(1, 2, 2)
fpr_crowded, tpr_crowded, _ = roc_curve(results_df_0["actual_crowded"], results_df_0["predicted_crowded"])
roc_auc_crowded = auc(fpr_crowded, tpr_crowded)
plt.plot(fpr_crowded, tpr_crowded, label=f"AUC = {roc_auc_crowded:.2f}")
plt.plot([0, 1], [0, 1], linestyle="dashed", color="red")  # Random chance line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for Crowded Prediction (only image as input)")
plt.legend()

plt.tight_layout()
plt.show()